# Test the attacker interface in a Cyberbattle environment

-   initialisation of the litlle environment
-   initialisation of profiles
-   initialisation of the attack wallet the attacker will be able to use
-   perform actions to discover the flag

In [1]:
from cyberbattle.agents.attacker.attacker_interface import Attacker, AttackerGoal

from cyberbattle.agents.battle_environment import CyberBattleEnv
from cyberbattle.env.samples.little_network import get_little_environment_network, Dev, DSI

num_client = 5
net = get_little_environment_network(num_client)
profiles = {
    DSI(num_client): 1,
    Dev(num_client): num_client - 1
}
env = CyberBattleEnv(net, profiles)

In [2]:
env.attacks.get_attacks_by_machines_string()

{'PC_1': {'Query Registry': {'data sources triggered': 'Process',
   'phase name': 'discovery',
   'Type': 'Local'}},
 'PC_2': {},
 'PC_3': {},
 'PC_4': {},
 'PC_5': {'Password Managers': {'data sources triggered': 'Process',
   'phase name': 'credential-access',
   'Type': 'Remote'}},
 'Switch_1': {},
 'Switch_2': {},
 'Router': {},
 'DatabaseServer': {'Browser Session Hijacking': {'data sources triggered': 'Process',
   'phase name': 'collection',
   'Type': 'Local'}},
 'MailServer': {'Network Service Scanning': {'data sources triggered': 'Cloud Service',
   'phase name': 'discovery',
   'Type': 'Local'}},
 'CommunicationServer': {},
 'GoogleDrive': {'Steal Application Access Token': {'data sources triggered': 'User Account',
   'phase name': 'credential-access',
   'Type': 'Remote'}},
 'Firewall_1': {},
 'Firewall_2': {}}

In [3]:
attacker = Attacker(
    goals=AttackerGoal(
        reward=0,
        nb_flag=1
    ),
    attacks=env.id_to_attack,
    network=net,
    attacks_by_machine=env.attacks_by_machine,
    start_time=env.get_start_time()
)

print(attacker.get_discovered_machines())
attacker.attack_as_string()

['PC_1']


{0: 'Query Registry',
 1: 'Password Managers',
 2: 'Browser Session Hijacking',
 3: 'Network Service Scanning',
 4: 'Steal Application Access Token'}

In [4]:
import numpy as np 

attack = {'local': np.array([0, 0])}
reward, activity = attacker.on_step(attack)

print(reward, activity.get_description())

20 Is activity : True, source : PC_1, target : PC_1, port : sudo, data source triggered : Process, is error : 0


In [5]:
attacker.get_discovered_machines()

['PC_1', 'PC_2', 'PC_3', 'PC_4', 'PC_5']

In [6]:
attack = {'local': np.array([0, 3])}
reward, activity = attacker.on_step(attack)

print(reward, activity.get_description())

-10 Is activity : False, source : None, target : None, port : None, data source triggered : None, is error : None


In [7]:
attack = {'local': np.array([0, 3])}
reward, activity = attacker.on_step(attack)

print(reward, activity.get_description())

-12 Is activity : False, source : None, target : None, port : None, data source triggered : None, is error : None


In [8]:
attack = {'remote': np.array([0, 4, 1])}
reward, activity = attacker.on_step(attack)

print(reward, activity.get_description())

25 Is activity : True, source : PC_1, target : PC_5, port : sudo, data source triggered : Process, is error : 0


In [9]:
attacker.get_discovered_machines(), attacker.get_discovered_credentials()

(['PC_1', 'PC_2', 'PC_3', 'PC_4', 'PC_5', 'MailServer'],
 [('HTTPS', 'MailServer', 'DSI')])

In [10]:
attack = {'connect': np.array([0, 0])}
reward, activity = attacker.on_step(attack)

print(reward, activity.get_description())

250 Is activity : True, source : PC_1, target : MailServer, port : HTTPS, data source triggered : User Account: User Account Authentification, is error : 0


In [11]:
attacker.get_infected_machines()

[('PC_1', 0.0), ('MailServer', 0.7100656032562256)]

In [12]:
attack = {'local': np.array([5, 3])}
reward, activity = attacker.on_step(attack)

print(reward, activity.get_description())

13 Is activity : True, source : MailServer, target : MailServer, port : HTTPS, data source triggered : Cloud Service, is error : 0


In [13]:
attacker.get_discovered_machines()

['PC_1',
 'PC_2',
 'PC_3',
 'PC_4',
 'PC_5',
 'MailServer',
 'DatabaseServer',
 'CommunicationServer',
 'GoogleDrive']

In [14]:
attack = {'remote': np.array([0, 5, 4])}
reward, activity = attacker.on_step(attack)

print(reward, activity.get_description())

-10 Is activity : False, source : None, target : None, port : None, data source triggered : None, is error : None


In [15]:
attack = {'remote': np.array([0, 8, 4])}
reward, activity = attacker.on_step(attack)

print(reward, activity.get_description())

18 Is activity : True, source : PC_1, target : GoogleDrive, port : HTTPS, data source triggered : User Account, is error : 0


In [16]:
attacker.get_discovered_machines(), attacker.get_discovered_credentials()

(['PC_1',
  'PC_2',
  'PC_3',
  'PC_4',
  'PC_5',
  'MailServer',
  'DatabaseServer',
  'CommunicationServer',
  'GoogleDrive'],
 [('HTTPS', 'MailServer', 'DSI'), ('HTTPS', 'DatabaseServer', 'DSI')])

In [17]:
attack = {'connect': np.array([0, 1])}
reward, activity = attacker.on_step(attack)

print(reward, activity.get_description())

1050 Is activity : True, source : PC_1, target : DatabaseServer, port : HTTPS, data source triggered : User Account: User Account Authentification, is error : 0


In [18]:
attacker.get_infected_machines()

[('PC_1', 0.0),
 ('MailServer', 0.7100656032562256),
 ('DatabaseServer', 1.2506592273712158)]

In [19]:
attack = {'submarine'}
reward, activity = attacker.on_step(attack)

print(reward, activity.get_description())

0 Is activity : False, source : None, target : None, port : None, data source triggered : None, is error : None


In [20]:
attack = {'local': np.array([6, 2])}
reward, activity = attacker.on_step(attack)

print(reward, activity.get_description())

1000 Is activity : True, source : DatabaseServer, target : DatabaseServer, port : HTTPS, data source triggered : Logon Session, is error : 0


In [21]:
attacker.get_captured_flag(), attacker.get_cumulative_rewards(), attacker.get_total_reward()

(1,
 array([  20,   10,   -2,   23,  273,  286,  276,  294, 1344, 2344],
       dtype=int32),
 2344)

In [22]:
attacker.reached_goals()

True

In [24]:
attacker.display_history(x='iteration')